In [1]:
import pandas as pd
import seaborn as sns
import os 
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# load data
df_dict = {}

for file in os.listdir("../final_data"):
   if file.endswith(".tsv"):
        filename = os.path.splitext(os.path.basename(file))[0]
        temp_df = pd.read_csv(os.path.join('../final_data', file), delimiter='\t')
        df_dict[filename] = temp_df

cost_df = df_dict['cost']

# since coffins column is missing values, add the coffin column from coffins.tsv
coffins_df = df_dict['coffins']
merged_df = cost_df.merge(coffins_df, left_on="coffins", right_on="total_cost", how="left")
cost_df["coffins"] = cost_df["coffins"].fillna(merged_df["coffins"])

# some of the missing values have different keys - filling in manually
# found this value in coffins_df
cost_df.at[0, "coffins"] = 590.0
# cannot find k&m in coffins_df, in paper, it says that Kha and Merit are similar to Yuya and Tuya, so filling their coffin value wih 590
cost_df.at[2, "coffins"] = 590.0

# filling in missing jewelry values
jewel_df = df_dict['jewel']
# putting 0 for iabtina because appendix in paper says there is no jewelry
cost_df.at[25, "jewlery"] = 0
# cannot find data for NuMan, putting 85 which is the average of the rank above and rank below cost on jewelry
cost_df.at[75, "jewlery"] = 85
cost_df = cost_df.rename(columns = {"jewlery" : "jewelry"})

# filling in missing values for profess
proff_df = df_dict['proff']
# no professional equipment for Sat-Re, filling in 0
cost_df.at[29, "profess"] = 0

# filling in missing values for toiletries 
# no information about toiletries for Khay, filling in the average of the rank above and below
cost_df.at[5, "toiletries"] = 13.5
# drop some repeat tombs
cost_df = cost_df[~cost_df['tomb'].isin(['Yuya','Kha', 'Merit', "Tuya", "T&A"])]

print(cost_df.isnull().sum())
cost_df = cost_df.drop(["rank"], axis=1)

tomb           0
rank           0
grand_total    0
amphorae       0
bouquet        0
bowls          0
boxbas         0
coffins        0
fertility      0
funerary       0
furniture      0
jars           0
jewelry        0
personal       0
profess        0
shabti         0
toiletries     0
vessels        0
dtype: int64


In [10]:
def assign_manual_label(row):
    tut = ["Tutankhamen"]
    elite = ["Y&T", "K&M", "Mahirper"]
    high_mid = ["Nakht", "Khay", "Hatnofer", "Neferkhewet", "Renofer", "Ruyu", "Boki", "Amenemheb", "Siamun", "Hentut'u", "Mahy", "Bakiset",
                "Taat", "Tahuty", "Ahotep", "Hatiay", "Setau", "S&N", "Petrie"]
    mid = ["Harmose", "Mentuhotep", "DeM 1389", "DeM 1381", "DeM 1380", "DeM 1377", "DeM 1375", "# 78 f", "# 53 m", "# 18 ?", "SR+", "Amenemhet", 
           "B+", "Ramose", "Maya", "Nub"]
    

    if row["tomb"] in tut:
        return "Tutankhamen"
    elif row["tomb"] in elite:
        return "Elite"
    elif row["tomb"] in high_mid:
        return "High-Middle"
    elif row["tomb"] in mid:
        return "Middle"
    else:
        return "Low"

# assign status from paper 
cost_df["status"] = cost_df.apply(assign_manual_label, axis=1)

In [11]:
# create group dataframes
elite_df = cost_df[cost_df["status"] == "Elite"]
high_mid_df = cost_df[cost_df["status"] == "High-Middle"]
mid_df = cost_df[cost_df["status"] == "Middle"]
low_df = cost_df[cost_df["status"] == "Low"]

df_list = [elite_df, high_mid_df, mid_df, low_df]


In [12]:
# PERMANOVA
from scipy.spatial.distance import pdist, squareform
from skbio.stats.distance import DistanceMatrix, permanova, permdisp

# scale features
no_tut = cost_df[cost_df['tomb'] != "Tutankhamen"]
feature_matrix = no_tut.drop(['tomb', 'status'], axis=1)
feature_matrix = StandardScaler().fit_transform(feature_matrix)

# calculate distance matrix
distances = pdist(feature_matrix, metric='euclidean')
dist_matrix = DistanceMatrix(squareform(distances), ids=no_tut["tomb"])

# permanova calculation
id_df = no_tut.set_index("tomb").loc[dist_matrix.ids, ["status"]]
pn = permanova(dist_matrix, grouping=id_df, column="status")
print(pn)
print('\n\n')



method name               PERMANOVA
test statistic name        pseudo-F
sample size                     133
number of groups                  4
test statistic            20.835505
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object





In [13]:
from rpy2.robjects import pandas2ri

os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"

pandas2ri.activate()

# Extract status column into a separate Series
group_vector = id_df["status"]

# Then send both to R
%R -i feature_matrix -i group_vector

In [14]:
%%R
library(vegan)

# Compute Bray-Curtis distances (or Euclidean as in your example)
dist_matrix <- vegdist(feature_matrix, method = "euclidean")

# Run PERMDISP (using centroids)
dispersion <- betadisper(dist_matrix, group_vector, type = "centroid")

# Test for differences in dispersion
permdisp_result <- anova(dispersion)
print(permdisp_result)


Analysis of Variance Table

Response: Distances
           Df Sum Sq Mean Sq F value    Pr(>F)    
Groups      3 546.02 182.008  69.656 < 2.2e-16 ***
Residuals 129 337.07   2.613                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [15]:
import rpy2.robjects as robjects

robjects.r('''
    library(vegan)
    dist_matrix <- vegdist(feature_matrix, method = "euclidean")
    dispersion <- betadisper(dist_matrix, group_vector, type = "centroid")
    result <- anova(dispersion)
    print(result)
''')


Analysis of Variance Table

Response: Distances
           Df Sum Sq

 Mean Sq F value    Pr(>F)    
Groups      3 546.02 182.008  69.656 < 2.2e-16 ***
Residuals 129 337.07   2.613                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
